# Small Language Model with Reinforcement Learning.
- The goal of this project is to demonstrate the importance of reinceforcement learning and its performance on small language model.

## Important tasks:
- Pretrain Small Language Model (10-100 million parameters)
- RLHF pipeline

### What is small language model (SLM)?
- The difference between small language model and large language model is in its training dataset, the transformer architecture remains same. It sole purpose is to pretrain language model for specific task whereas LLM can do multiple tasks. Thereby a good quality dataset is important for our SLM to capture the grammer and context of a sentence and the task.

### Dataset
- For the simplicity, we load the dataset of tiny stories mentioned in this paper https://arxiv.org/abs/2305.07759, we can find the dataset from hugging face.

### Step 1: Import Dataset
TinyStories is a synthetic dataset of short stories that only contain words that a typical 3 to 4-year-olds usually understand, generated by GPT-3.5 and GPT-4. We can get it from HuggingFace.

In [1]:
!pip install datasets

In [2]:
from datasets import load_dataset

In [3]:
ds = load_dataset("roneneldan/TinyStories")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00004-2d5a1467fff108(…):   0%|          | 0.00/249M [00:00<?, ?B/s]

data/train-00001-of-00004-5852b56a2bd28f(…):   0%|          | 0.00/248M [00:00<?, ?B/s]

data/train-00002-of-00004-a26307300439e9(…):   0%|          | 0.00/246M [00:00<?, ?B/s]

data/train-00003-of-00004-d243063613e5a0(…):   0%|          | 0.00/248M [00:00<?, ?B/s]

data/validation-00000-of-00001-869c898b5(…):   0%|          | 0.00/9.99M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2119719 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/21990 [00:00<?, ? examples/s]

- Peek into the dataset

In [4]:
print('Total Number of training data', len(ds['train']))
print('Total Number of validation data', len(ds['validation']))
print("\n============Training Sample==============\n")
print(ds['train'][0])
print("\n============Validation Sample============\n")
print(ds['validation'][0])

Total Number of training data 2119719
Total Number of validation data 21990

============Training Sample==============

{'text': 'One day, a little girl named Lily found a needle in her room. She knew it was difficult to play with it because it was sharp. Lily wanted to share the needle with her mom, so she could sew a button on her shirt.\n\nLily went to her mom and said, "Mom, I found this needle. Can you share it with me and sew my shirt?" Her mom smiled and said, "Yes, Lily, we can share the needle and fix your shirt."\n\nTogether, they shared the needle and sewed the button on Lily\'s shirt. It was not difficult for them because they were sharing and helping each other. After they finished, Lily thanked her mom for sharing the needle and fixing her shirt. They both felt happy because they had shared and worked together.'}

============Validation Sample============

{'text': 'Spot. Spot saw the shiny car and said, "Wow, Kitty, your car is so bright and clean!" Kitty smiled and repl

### Tokenize Dataset
- In this section tokenize the dataset into tokenIDs

In [6]:
!pip install tiktoken
import tiktoken

encoder = tiktoken.get_encoding("gpt2")

# process function takes a sample from the dataset and returns ids and length of ids
def process(sample):
    text = sample['text']
    ids = encoder.encode_ordinary(text) # encode_ordinary to avoid adding special tokens
    return {'ids': ids, 'len': len(ids)}

tokenized = ds.map(process, remove_columns=['text'], num_proc=4)

Map (num_proc=4):   0%|          | 0/2119719 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/21990 [00:00<?, ? examples/s]

In [7]:
print(type(tokenized))

<class 'datasets.dataset_dict.DatasetDict'>


In [9]:
tokenized['train'][0]

{'ids': [3198,
  1110,
  11,
  257,
  1310,
  2576,
  3706,
  20037,
  1043,
  257,
  17598,
  287,
  607,
  2119,
  13,
  1375,
  2993,
  340,
  373,
  2408,
  284,
  711,
  351,
  340,
  780,
  340,
  373,
  7786,
  13,
  20037,
  2227,
  284,
  2648,
  262,
  17598,
  351,
  607,
  1995,
  11,
  523,
  673,
  714,
  34249,
  257,
  4936,
  319,
  607,
  10147,
  13,
  198,
  198,
  43,
  813,
  1816,
  284,
  607,
  1995,
  290,
  531,
  11,
  366,
  29252,
  11,
  314,
  1043,
  428,
  17598,
  13,
  1680,
  345,
  2648,
  340,
  351,
  502,
  290,
  34249,
  616,
  10147,
  1701,
  2332,
  1995,
  13541,
  290,
  531,
  11,
  366,
  5297,
  11,
  20037,
  11,
  356,
  460,
  2648,
  262,
  17598,
  290,
  4259,
  534,
  10147,
  526,
  198,
  198,
  41631,
  11,
  484,
  4888,
  262,
  17598,
  290,
  384,
  19103,
  262,
  4936,
  319,
  20037,
  338,
  10147,
  13,
  632,
  373,
  407,
  2408,
  329,
  606,
  780,
  484,
  547,
  7373,
  290,
  5742,
  1123,
  584,
  13,
  2293,